<a href="https://colab.research.google.com/github/OGPanther08/ICP-7/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit_learn

In [ ]:
# Load model and predict on new data
def predict_sentiment(text):
    loaded_model = load_model("sentiment_model.h5")
    text = tokenizer.texts_to_sequences([text])
    text = pad_sequences(text, maxlen=X.shape[1])
    prediction = loaded_model.predict(text)
    sentiment = labelencoder.inverse_transform([prediction.argmax()])
    return sentiment[0]

# Example prediction
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @Donald Trump"
print("Predicted Sentiment:", predict_sentiment(new_text))

Best model saved as 'sentiment_model.h5'.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted Sentiment: Negative


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import re

# Load and preprocess data
data = pd.read_csv('/content/Sentiment (3) (2) (1).csv')
data = data[['text', 'sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

# Clean data
for idx, row in data.iterrows():
    row.iloc[0] = row.iloc[0].replace('rt', ' ')

# Tokenization and padding
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Encode labels
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Define input dimension based on training data
input_dim = X_train.shape[1]

# Define model creation function with tunable hyperparameters
def create_model(dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_dim,)))  # Use defined input_dim
    model.add(Dropout(dropout_rate))
    model.add(Dense(y.shape[1], activation='softmax'))  # Use softmax for multi-class output
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Apply GridSearchCV to tune batch size, epochs, optimizer, and dropout_rate
model = KerasClassifier(model=create_model, dropout_rate=0.2)

# Define the hyperparameters for tuning
#I would add more to this list, but I already feel stressed for time
param_grid = {
    'dropout_rate': [0.0,0.25,0.5],
    'epochs': [10,30,50],
    'batch_size': [5,20,35],
    'optimizer': ['adam', 'sgd', 'rmsprop']
}

# Set up GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid.fit(X_train, Y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Use the best model
best_model = grid_result.best_estimator_.model_

# Evaluate the best model on the test data
score, acc = best_model.evaluate(X_test, Y_test, verbose=2)
print("Test Score:", score)
print("Test Accuracy:", acc)

# Save the model
best_model.save("sentiment_model.h5")
print("Best model saved as 'sentiment_model.h5'.")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4946 - loss: 49.7725
Epoch 2/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5154 - loss: 17.0213
Epoch 3/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5248 - loss: 10.7043
Epoch 4/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5345 - loss: 8.1807
Epoch 5/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5358 - loss: 6.1243
Epoch 6/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5303 - loss: 4.3951
Epoch 7/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5225 - loss: 3.5704
Epoch 8/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5302 - loss: 2.8564
Epoch 9/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5365 - loss: 2.1320
Epoch 10/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5252 - loss: 1.7312
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4549 - loss: 57.8492
Epoch 2/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4947 - loss: 16.0846
Epoch 3/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5003 - loss: 10.6671
Epoch 4/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5092 - loss: 7.4109
Epoch 5/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5201 - loss: 5.2986
Epoch 6/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5095 - loss: 4.4164
Epoch 7/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5120 - loss: 3.6773
Epoch 8/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5219 - loss: 2.5765
Epoch 9/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5286 - loss: 2.0635
Epoch 10/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5462 - loss: 1.5289
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/10


...

178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4247 - loss: 164.0570
Epoch 2/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4751 - loss: 61.3201
Epoch 3/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4619 - loss: 29.0374
Epoch 4/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4776 - loss: 13.4889
Epoch 5/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4927 - loss: 5.6390
Epoch 6/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4749 - loss: 3.3002
Epoch 7/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5061 - loss: 2.2516
Epoch 8/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5311 - loss: 1.7082
Epoch 9/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5491 - loss: 1.3422
Epoch 10/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5372 - loss: 1.5626
Epoch 11/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5307 - loss: 1.4933
Epoch 12/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1859/1859 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4633 - loss: 50.2937
Epoch 2/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.4885 - loss: 11.1975
Epoch 3/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5126 - loss: 5.9424
Epoch 4/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5002 - loss: 4.2823
Epoch 5/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.5221 - loss: 2.7108
Epoch 6/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5137 - loss: 1.9641
Epoch 7/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5469 - loss: 1.4370
Epoch 8/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5681 - loss: 1.1221
Epoch 9/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6071 - loss: 0.9810
Epoch 10/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6125 - loss: 0.9341
Epoch 11/30
1859/1859 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6215 - loss: 0.9208
Epoch 12/30
1859/1859 ━━━━━━━━━━━━━━━━━

Test Score: 1.078109622001648
Test Accuracy: 0.6026649475097656
Best model saved as 'sentiment_lstm_model.h5'.


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'sentiment_lstm_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Sentiment (3) (2) (1).csv to Sentiment (3) (2) (1).csv


In [ ]:
!pip install scikeras